## Getting Started with prompt Template and Chat Prompt Template

#### Use PromptTemplate to create a template for a string prompt.
By default, PromptTemplate uses Python’s str.format syntax for templating.

In [4]:
%pip install langchain python-dotenv


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 1.7 MB/s eta 0:00:002.0 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.5/814.5 kB 4.9 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.4/278.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.1 MB/s eta 0:00:00m eta 0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 537.1 kB/s eta 0:00:00MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 855.2 kB/s eta 0:00:001m102.8 MB/s eta 0:00:01
  Created wheel for greenlet: filename=greenlet-3.0.3-cp312-cp312-maco

In [5]:
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_community.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from dotenv import load_dotenv

In [9]:
load_dotenv()

True

In [10]:
model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.5)

### A simple string based Prompt formatting

In [11]:
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt = prompt_template.format(adjective="sad", content="British Weather")

response = model.invoke(prompt)
print(response)



Why did the British weather forecasters feel guilty?

Because they always make promises they can't keep.


### ChatPromptTemplate: The prompt to chat models is a list of chat messages.
Each chat message is associated with content, and an additional parameter called role. <br>
For example, in the OpenAI Chat Completions API, a chat message can be associated with an AI assistant, a human or a system role.

In [12]:
from langchain_core.prompts import ChatPromptTemplate

In [13]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)
prompt = chat_template.format_messages(name="Bob", user_input="What is your name and what are you good at?")
response = model.invoke(prompt)
print(response)


AI: My name is Bob and I am a helpful AI bot. I am good at providing information, answering questions, and assisting with tasks. Is there something specific you need help with?


### Various ways of formatting System/Human/AI prompts

In [14]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
prompt = chat_template.format_messages(text="I don't like eating tasty things")

response = model.invoke(prompt)
print(response)


System: I may not always agree with your taste buds, but I believe that there are plenty of delicious foods out there waiting for you to discover and enjoy!


### Providing a Context along with the Prompt

In [15]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: """

In [16]:
print(model.invoke(prompt))

Hugging Face, OpenAI, and Cohere offer LLMs.


### Langchain’s ```FewShotPromptTemplate``` caters to source knowledge input. The idea is to “train” the model on a few examples — we call this few-shot learning — and these examples are given to the model within the prompt.

The goal of few-shot prompt templates are to dynamically select examples based on an input, and then format the examples in a final prompt to provide for the model.<br>

<b>Fixed Examples</b><br>
The most basic (and common) few-shot prompting technique is to use a fixed prompt example. This way you can select a chain, evaluate it, and avoid worrying about additional moving parts in production. <br>

The basic components of the template are: - examples: A list of dictionary examples to include in the final prompt. - example_prompt: converts each example into 1 or more messages through its format_messages method. A common example would be to convert each example into one human message and one AI message response, or a human message followed by a function call message.

In [17]:
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    },
    {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }
]

# create a example template
example_template = """
User: {query}
AI: {answer}
"""

In [19]:
# create a prompt example from above template

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

In [20]:
# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [21]:
query = "What movie should I watch this evening?"

print(few_shot_prompt_template.format(query=query))

The following are excerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples:



User: How are you?
AI: I can't complain but sometimes I still do.



User: What time is it?
AI: It's time to get a watch.



User: What movie should I watch this evening?
AI: 


In [22]:
chain = few_shot_prompt_template | model

chain.invoke({"query": query})

' How about "The Shawshank Redemption"? Or "The Godfather"? Oh wait, you said movie, not masterpiece. Maybe try "Sharknado" for some mindless entertainment. '